# The Game

In [1]:
import numpy as np
import time
import keyboard
from IPython.display import clear_output

class Snake:
    """ Snake Game
    
    :property tick: the current tick
    :property player: the player that determines the next move
    :property dimx: x size of field
    :property dimy: y size of field
    :property direction: the direction can have 4 values:
        1 -> left
        2 -> up
        3 -> right
        4 -> down
    :property snake: the coordinates of the snake parts
    :property snake_max: max length of the snake
    :property food: the coordinates of the food
    """
    
    def __init__(self, player, dim=(60,30)):
        """ Initializes the game. 
        """
        
        self.tick = 0
        self.player = player
        self.dimx, self.dimy = dim
        self.direction = 2
        self.snake = [(self.dimy // 2, self.dimx // 2)]
        self.snake_max = 1
        self.food = (5,5)
        
    def run(self):
        """ The function which runs the game. Contains all the game logic.
        """
        
        # necessary to make the start of the game less awkward
        self._display_state()
        print("Press 'w' key to start!")
        while True:
            key = keyboard.read_key()
            if key == "w":
                break
        
        # game loop
        while True:
            # advance game state by 1 tick
            self.tick += 1
            
            # get next move
            next_move = self.player.get_next_move((self.dimx, self.dimy), self.snake, self.food)
            
            # only move every 10 ticks
            if self.tick == 10:
                # process next move
                if not self._process_move(next_move):
                    break

                # display updated state and reset ticks
                self._display_state()
                self.tick = 0
            
            # sleep
            time.sleep(0.01)
        
        # game is finished
        print("Failed!")
            
    def _process_move(self, move):
        """ Checks the validity of the given move and proceeds accordingly.
        
        :parameter move: next direction (if valid)
        
        
        :return game_not_finished: determines whether the game is finished or not
        """
        
        # check if move is valid
        if move == 1 or move == 2 or move == 3 or move == 4:
            self.direction = move
        
        # determine new position of snake head
        y, x  = self.snake[-1]
        x += self._new_x(self.direction)
        y += self._new_y(self.direction)
        
        # check for colision with border
        if x < 1 or x == self.dimx-1 or y < 1 or y == self.dimy-1:
            return False
        
        # check for collision with food
        if (y,x) == self.food:
            self.snake_max += 1
            self.food = (5,5)
            
        # check for collision with snake body
        for i in self.snake:
            if i == (y,x):
                return False
        
        # set new position of head
        self.snake.append((y, x))
        if self.snake_max < len(self.snake):
            del self.snake[0]
        
        return True
    
    def _new_x(self, move):
        """ Maps the given move to a modification to x coordinate. E.g. 2 = up which means x 
            will not change
        
        :parameter move: the next direction to go in
        """
        
        if move == 1:
            return -1
        elif move == 2:
            return 0
        elif move == 3:
            return 1
        elif move == 4:
            return 0
    
    def _new_y(self, move):
        """ Maps the given move to a modification to y coordinate. E.g. 2 = up which means y 
            will decrease by 1
        
        :parameter move: the next direction to go in
        """
        
        if move == 1:
            return 0
        elif move == 2:
            return -1
        elif move == 3:
            return 0
        elif move == 4:
            return 1
        
    def _display_state(self):
        """ Clears the output and prints the current state of the game.
        """
        
        # clear output and prepare an empty board
        clear_output(wait=True)
        field_to_draw = np.zeros((self.dimy, self.dimx), dtype=np.int8)
        
        # draw snake
        for i in self.snake:
            y, x = i
            field_to_draw[y, x] = 9
            
        # draw food
        x, y = self.food
        field_to_draw[y,x] = 5
        
        # draw field details (including snake, food)
        game = ""
        for i in range(field_to_draw.shape[0]):
            for j in range(field_to_draw.shape[1]):
                
                # border
                if j == 0 or j == self.dimx-1:
                    game = game + "|"
                    continue
                elif i == 0 or i == self.dimy-1:
                    game = game + "-"
                    continue
                
                # snake, food and empty cells
                symbol = field_to_draw[i, j]
                if symbol == 0:
                    game = game + " "
                else:
                    game = game + str(symbol)
                
            game = game + "\n"
        # print the whole field
        # (this increases performance since print is only called once)
        print(game)
            
class HumanPlayer:
    """ A Player class that can be used to play the Snake game.
    """
    
    def __init__(self):
        self.last_key = -1
        
    def get_next_move(self, field_dims, snake, food):
        """ Determines the next move of the snake according to player input via keyboard.
        """

        if keyboard.is_pressed("a"):
            self.last_key = 1
            return 1
        elif keyboard.is_pressed("w"):
            self.last_key = 2
            return 2
        elif keyboard.is_pressed("d"):
            self.last_key = 3
            return 3
        elif keyboard.is_pressed("s"):
            self.last_key = 4
            return 4
        
        return self.last_key

In [3]:
snake = Snake(HumanPlayer())
snake.run()

|----------------------------------------------------------|
|                                                          |
|                                                          |
|                                                          |
|                                                          |
|    5                                                     |
|                                                          |
|                                                          |
|                                                          |
|   9999                                                   |
|   9  9                                                   |
|   9999                                                   |
|                                                          |
|                                                          |
|                                                          |
|                                                          |
|                       